# Error bars

Given the formula that links the differential cross section to the event rate measured by our experiment:

$$
\frac{dR}{d \Omega} = \epsilon_{spectrometer}\left(\theta \right) \cdot \frac{n_{gate}}{T} \cdot n_{c} \cdot \wp(\theta; \lambda', \lambda '') \cdot \frac{d \sigma}{d \Omega}
$$

it is crucial to correctly evaluate the uncertainities of these measurements. 

## Statistical and systematical uncertainties + work flow
First thing first it is fundamental to distinguish between the various sources of errors one can come across during a measurement in a nuclear physics' experiment.
In particular we need to separate the contributions given by $\textit{statistical}$ and $\textit{systematical}$ uncertainties:

- Statistical uncertainities could come from the counting of photons or the errors associated to the fit parameters, a square sum will be used to take account of them.
- Systematical uncertainties arise from various effects, such as the shift of the Compton peak, the errors associated with the geometrical measurements of the apparatus and so on, we'll take account of them with a square sum as well.

By looking at the expression above one could come up with an expression for the uncertainty that looks like:

$$
\left(\frac{\delta R}{R} \right) = \left(\frac{\delta \epsilon_{spectrometer}}{\epsilon_{spectrometer}} \right) \oplus \left(\frac{\delta n_{gate}}{n_{gate}} \right) \oplus \left(\frac{\delta n_c}{n_c} \right) \oplus \left(\frac{\delta \wp}{\wp} \right) 
$$

assuming every quantity considered in this expression is independent from one another and has a relatively small error associated to it.

To make sure randomness and bias are not mixed together these two kinds of uncertainties will be combined as:

$$
\sigma_{tot} = \sigma_{statistic} + \sigma_{systematic}
$$

so that one can split the two contributes in:

$$ \left(\frac{\delta Q_{stat}}{Q} \right)^2 = \sum_{X \, \in \, (\epsilon_{spectr.}, n_{gate}, T, n_c, \wp)}  \left(\frac{\delta X_{stat}}{X} \right)^2 $$
$$ \left(\frac{\delta Q_{sist}}{Q} \right)^2 = \sum_{X \, \in \, (\epsilon_{spectr.}, n_{gate}, T, n_c, \wp)}  \left(\frac{\delta X_{sist}}{X} \right)^2 $$


## $n_c$ uncertainty

Given:

$$
n_c = \rho \frac{N_a \cdot Z}{MM}
$$

where: 
- $\rho$ is the density of the scattering target
- $N_a$ is the Avogadro number
- $\text{MM}$ is the molar mass of the scattering target

One can clearly see that all these quantities are known with extreme accuracy, we won't take account of their uncertainties.

## $\epsilon_{spectrometer}$ uncertainty

In this case the value of $\epsilon_{spectrometer}$ is obtained by an interpolation, the error associated to it will then be consider as a statistical uncertainty. Given the formula used to fit the efficiency curve: 

$$
\epsilon_{spectrometer} = A \cdot E^{-B} \cdot \exp(-C \cdot E) + D
$$

where $A,B,C,D$ are all model parameters and $E$ is the energy of the incoming gamma ray. Given that, one can compute the error associated to this quantity as:

$$
\delta \epsilon = \left( \frac{\partial \epsilon}{\partial A} \cdot \delta A \right) \oplus \left( \frac{\partial \epsilon}{\partial B} \cdot \delta B \right) \oplus \left( \frac{\partial \epsilon}{\partial C} \cdot \delta C \right) \oplus \left( \frac{\partial \epsilon}{\partial D} \cdot \delta D \right)
$$

$$
\delta \epsilon = \exp(-E \cdot C) \left( \delta A \cdot E^{-B} - \frac{\delta B \cdot A \cdot B}{E^{B+1}} - \frac{A \cdot C \cdot \delta C }{E^{B}} \right) + \delta D
$$

 SBAGLIATA IMO

$$
\delta \epsilon = \sqrt{\left[E^{-B} \cdot e^{-CE} \cdot \delta A \right]^2 + \left[A \cdot E^{-B} \cdot e^{-CE} \cdot \ln(B) \cdot \delta B \right]^2 + \left[A \cdot E^{-B} \cdot e^{-CE} \cdot E \cdot \delta C \right]^2 + \left[ \delta D \right]^2}
$$

Parameters from the efficency fit:

  $$ A = 1.55710 ± 2.91779 $$
 $$ B = -0.09831 ± 1.03314  $$
 $$ C = 3.53226 ± 2.39943 $$
 $$ D = 0.10209 ± 0.02355 $$


In [50]:
import numpy as np

# Parametri del fit e le relative incertezze
A = 1.55710
delta_A = 2.91779

B = -0.09831
delta_B = 1.03314

C = 3.53226
delta_C = 2.39943

D = 0.10209
delta_D = 0.02355

def epsilon_spettrometer(theta): 
    A = 1.55710
    B = -0.09831
    C = 3.53226
    D = 0.10209
   
    E  = 511/(2 - np.cos(theta)) * 1e-3 #da MeV a keV

    return A * pow(E, -B) * np.exp(-C * E) + D

'''def delta_eff(E):
    # Calcolo del termine esponenziale
    exp_term = np.exp(-E * C)
    
    # Calcolo dei singoli contributi all'incertezza
    term1 = delta_A * E**(-B)
    term2 = (delta_B * A * B) / (E**(B + 1))
    term3 = (A * C * delta_C) / (E**B)

    # Formula completa
    delta_epsilon = exp_term * (term1 - term2 - term3) + delta_D
    return delta_epsilon
'''

def delta_eff(E):
    term1 = E**(-2 * B) * np.exp(-2 * C * E) * delta_A**2
    term2 = A**2 * E**(-2 * B) * np.exp(-2 * C * E) * (np.log(E))**2 * delta_B**2
    term3 = A**2 * E**(-2 * B) * np.exp(-2 * C * E) * (E**2) * delta_C**2 
    term4 = delta_D**2

    delta_epsilon = np.sqrt(term1 + term2 + term3 + term4)
    return delta_epsilon

E_input = 511 
result = delta_eff(E_input)
print(f"\nIncertezza sull'efficienza (δϵ) a E = {E_input:.2f} è {result:.6f}")



Incertezza sull'efficienza (δϵ) a E = 511.00 è 0.023550


## $n_{gate}$ uncertainty

Given: 

$$
n_{gate} = 2 \cdot S \left(t \right) \cdot \text{BR} \cdot \frac{\Delta\Omega}{4\pi} \cdot \epsilon_{gate}(511)
$$

where: 
- $S(t)$ is the activity of the source in Bq
- $\text{BR}$ is the branching ratio of the 511 keV photon
- $\frac{\Delta\Omega}{4\pi}$ is the solid angle
- $\epsilon_{gate}(511)$ is the efficiency of the gate detector for the 511 keV photon

In this case one can assume that the branching ratio (BR) is well known, the uncertainties linked to $S \left(t \right)$ and $\epsilon_{gate}$ are statistical errors and the one associated to $\Delta \Omega$ is systematical.

### $\delta S(t)$ computation
For this uncertainty, an error of 0.5 cm in the distance from the detector and 3 mm in the displacement from the detector axis was considered. The acquisition times were assumed to be error-free, as they were set within the Maestro application. For the number of counts, the uncertainty associated with Poisson statistics was taken into account.
The associated error is then:

$$
\delta S \left( t \right) = 11647 \ Bq.
$$

### $\delta \epsilon_{gate}$ computation

For this computation please refer to the previous section.
We impose a 5% error on each point and from the linear fit we gain:

$$
\delta \epsilon_{gate} (511) = 0.03344  
$$

### $\delta (\Delta \Omega / 4\pi)$ computation

For the uncertainty linked to the measurement of the solid angle covered by the spectrometer one can start from the formula:

$$
\frac{\Delta\Omega}{4\pi} =  \frac{1 - \cos (\beta)}{2}
$$

where $\beta$ is computed as: 

$$
\beta = \arctan \left( \frac{r_{gate}}{d_{source-gate}} \right).
$$

Both of these two measurements are known with their uncertainty, $r_{gate} = (1,27 \pm 0,02) \, cm$ and ${d_{source-gate}} = d =(16.0 \pm 0.1) \, cm$, knowing that one can propagate the errors using the error propagation expression.

$$
\left(\delta (\Delta \Omega / 4\pi) \right) = \left( \frac{\partial (\Delta \Omega / 4\pi)}{\partial r} \delta r\right) \oplus \left( \frac{\partial (\Delta \Omega / 4\pi)}{\partial d} \delta d\right)
$$

and that:

$$
\frac{\partial (\Delta \Omega / 4\pi)}{\partial r} = \frac{r}{2 d^2 \cdot \left( \frac{r^2}{d^2} + 1 \right)^{3/2}}
$$

$$
\frac{\partial (\Delta \Omega / 4\pi)}{\partial d} = - \frac{r^2}{2 d^3 \cdot \left( \frac{r^2}{d^2} + 1 \right)^{3/2}}
$$

one can find the uncertainty asssociated to the solid angle using the formula below:

$$
\delta \left(\Delta \Omega / 4\pi \right) = \sqrt{\left(\frac{r}{2 d^2 \cdot \left( \frac{r^2}{d^2} + 1 \right)^{3/2}} \cdot \delta r \right)^2 + \left(\frac{r^2}{2 d^3 \cdot\left( \frac{r^2}{d^2} + 1 \right)^{3/2}} \cdot \delta d \right)^2}
$$


In [51]:
import math

# Valori noti (convertiti in metri se vuoi usare il SI, qui rimangono in cm)
r = 1.27       # cm
delta_r = 0.02 # cm
d = 18.54       # cm
delta_d = 0.1  # cm

## Denominatore comune
#denominator = (r**2 + d**2)**(3/2)
#
# Derivate parziali moltiplicate per incertezze
#term_r = (0.5 * r * d / denominator) * delta_r
#term_d = (0.5 * r**2 / denominator) * delta_d
#
## Errore totale
#delta_Omega = math.sqrt(term_r**2 + term_d**2)

term1  = r * delta_r / (2 * d**2 * (r**2/d**2 + 1)**(3/2))
term2 = r**2 * delta_d / (2 * d**3 * (r**2/d**2 + 1)**(3/2))

delta_Omega = np.sqrt(term1**2 + term2**2)

print(f"Incertezza sulla frazione di angolo solido ΔΩ/4π: {delta_Omega:.5e}")


Incertezza sulla frazione di angolo solido ΔΩ/4π: 3.87812e-05


## Final results

Knowing all the information we need, the uncertainty on the rate measurement can be expressed as:

$$
\frac{\delta R}{R} = \left[ \left(\frac{\delta \epsilon_{spectrometer}}{\epsilon_{spectrometer}} \right) \oplus \left(\frac{\delta S(t)}{S(t)} \right) \oplus \left(\frac{\delta \epsilon_{gate}}{\epsilon_{gate}} \right) \right] \pm \left( \frac{\delta \Delta \Omega}{\Delta \Omega} \right)
$$



In [52]:
def leggi_dati(file_path):
    # Inizializza liste per ogni colonna
    angle, rate, err_rate = [], [], []
    count, err_count = [], []
    channel, err_channel = [], []
    sigma, err_sigma = [], []

    with open(file_path, 'r') as file:
        lines = file.readlines()

        # Salta l'intestazione
        for line in lines[1:]:
            valori = line.strip().split()
            if len(valori) != 9:
                continue  # Salta righe non valide

            angle.append(float(valori[0]))
            rate.append(float(valori[1]))
            err_rate.append(float(valori[2]))
            count.append(float(valori[3]))
            err_count.append(float(valori[4]))
            channel.append(float(valori[5]))
            err_channel.append(float(valori[6]))
            sigma.append(float(valori[7]))
            err_sigma.append(float(valori[8]))

    return angle, rate, err_rate, count, err_count, channel, err_channel, sigma, err_sigma

file_path = "../Codes/data_analysis/parameters_pol4_riflection.txt"

angle, rate, err_rate, count, err_count, channel, err_channel, sigma, err_sigma = leggi_dati(file_path)

file_path = "../Codes/data_analysis/parameters_pol4_trasmission.txt"

angle_trasm, rate_trasm, err_rate_trasm, count_trasm, err_count_trasm, channel_trasm, err_channel_trasm, sigma_trasm, err_sigma_trasm = leggi_dati(file_path)


In [ ]:
# Dati tabella: angoli in gradi e valori corrispondenti
angles_deg = np.array([
    140, 135, 130, 125, 120, 115, 110, 105, 100, 95,
    90, 85, 80, 75, 70, 65, 60, 55, 50, 45,
    40, 35, 30, 25, 20
])

#N.B. PER IL 35 HO USATO IL VALORE IN TRASMISSIONE A 45 CM 
values = np.array([
    0.03244310169, 0.03244310169, 0.03249877382, 0.03249877382, 0.03249877382,
    0.03244310169, 0.03244310169, 0.03249877382, 0.03249877382, 0.03249877382,
    0.03244310169, 0.03244310169, 0.03244310169, 0.03244310169, 0.03244310169,
    0.03255449356, 0.03255449356, 0.03261026089, 0.03261026089, 0.0327778485,
    0.03288981157, 0.03300196504, 0.03322684314, 0.03345248278, 0.03384918453,
    0.03436265618, 0.03522700787, 0.03669138374, 0.03964779781, 0.04777407735
]) 

def angle_spectrometer(theta):
    # Converti radianti in gradi
    angle_deg = np.degrees(theta)
    
    # Trova indice del valore con angolo più vicino
    idx = (np.abs(angles_deg - angle_deg)).argmin()
    
    # Restituisci il valore corrispondente
    return 0.027

# Calcola i valori corrispondenti di angle_spectrometer per ogni angolo
spectrometer_values = np.array([angle_spectrometer(theta) for theta in angle])

# Dividi ogni rate per il corrispondente valore di angle_spectrometer
corrected_rates = rate / spectrometer_values



angle_trasm_25 = 0.02686251355
angle_trasm_45 = 0.009017319376

spectrometer_values_trasm = np.array([angle_trasm_45, angle_trasm_25, angle_trasm_45]) # rispettivamente 35 ,40 ,50 deg

corrected_rates_trasm = rate_trasm / spectrometer_values_trasm

In [54]:
S = 188900
dS = 11647

E = 0.14700
dE = 0.03344

Omega = 0.001567
dOmega = 1.09204e-04

angles = np.radians(angle)
energies = 511/(2 - np.cos(angles))
efficiencies = epsilon_spettrometer(angles)
err_tot = corrected_rates * (
    np.sqrt((dS / S)**2 + (dE / E)**2 + (delta_eff(energies) / efficiencies)**2) + dOmega / Omega
)
print(err_tot)




angles_trasm = np.radians(angle_trasm)
energies_trasm = 511/(2 - np.cos(angles_trasm))
efficiencies_trasm = epsilon_spettrometer(angles_trasm)
err_tot_trasm = corrected_rates_trasm * (
    np.sqrt((dS / S)**2 + (dE / E)**2 + (delta_eff(energies_trasm) / efficiencies_trasm)**2) + dOmega / Omega
)

print(err_tot_trasm)


[0.17644715 0.50003994 0.41596361 0.28369541 0.16201258 0.30069534
 0.14878624 0.16805405]
[0.79244875 0.23070609 0.4886342 ]


# Energy uncertainties

On the x-axis of our Klein-Nishina plot we put the scattering angles, linked to these quantities there are a few uncertainty factors such as:
- The shift in the position of the Compton peak due to temperature instability and other environmental factors.
- The error associated to the measurement of the scattering angle. 
- The uncertainty linked to the spectrometer resolution, which, of course, is not ideal.
- The statistic error derived from the gaussian fit of the Compton peak.

While these first three factors have a systematic nature, the last one can be considered as strictly statistical. Knowing this it is crucial to transform every energy-related quantity into an angle-related quantity, one can do that simply by using the inverted Compton formula shown below:

$$
\theta (E') = \arccos \left[2- \frac{511}{E'} \right]
$$

that gives rise to the associated uncertainty formula, which is:

$$
\delta \theta = \left |\frac{511}{E'^2 \cdot \sqrt{1 - \left(2 - \frac{511}{E'} \right)^2}} \right | \cdot \delta E' 
$$







## Peak shift uncertainty

Knowing that the Compton peak undergoes a mean fluctuation called $\Delta E_{peak} = 8 \, ch = 24 \, keV$, we used $\delta E_{peak} = \frac{\Delta E_{peak}}{2} = 4 \, ch = 12 \,keV$ to quantify our peak energy uncertainty. This can be expressed as:

$$
\delta \theta_{peak} = \left |\frac{511}{E'^2 \cdot \sqrt{1 - \left(2 - \frac{511}{E'} \right)^2}} \right | \cdot \delta E_{peak}  .
$$

In [55]:
dE = 12 #keV
dThetaPeak =np.array([])

for i in range(len(energies)):
    dTheta = 511/(energies[i]**2 * np.sqrt(1 - (2 - 511/energies[i])**2)) * dE
    dThetaPeak = np.append(dThetaPeak, dTheta)
print(dThetaPeak)

dThetaPeak_trasm = np.array([])
for i in range(len(energies_trasm)):
    dTheta = 511/(energies_trasm[i]**2 * np.sqrt(1 - (2 - 511/energies_trasm[i])**2)) * dE
    dThetaPeak_trasm = np.append(dThetaPeak_trasm, dTheta)
print(dThetaPeak_trasm)

[0.0556278  0.05646795 0.06101157 0.06869625 0.07953857 0.09393346
 0.11266458 0.13707422]
[0.0570896  0.0556278  0.05646795]


## Scattering angle uncertainty

This is chosen besed on the instrumental error used to measure $\theta_{scattering}$, and it is called $\delta \theta_{scattering}$.

Assuming $a = 5 \, deg$ one can compute the error assocaited to the placement of the detector at a given scattering angle as:

$$
\delta \theta_{scattering} = \frac{a}{\sqrt{12}}
$$


In [56]:
dThetaScattering = (5 * np.pi/180) / np.sqrt(12)
print(dThetaScattering)

0.02519165783658636


## Resolution uncertainty

The error due to the resolution of a detector can be expressed by means of the percentile resolution of our spectrometer, that can be computed as:

$$
\delta_{resolution} = \frac{2.35 \cdot \sigma_{fit}}{channel} \cdot 100
$$

Just like the previous case, this quantity needs to be expressed in terms of the scattering angle, this can be computed using:

$$
\delta \theta_{resolution} = \left| \frac{1}{sin^2 \theta \cdot (E')^2}\right| \cdot \delta_{resolution} 
$$

In [57]:

dThetaRis = 1/(np.sin(angles)**2 * (energies**2)) * err_sigma * 2.35 / angles * 100
print(dThetaRis)

dThetaRis_trasm = 1/(np.sin(angles_trasm)**2 * (energies_trasm**2)) * err_sigma_trasm * 2.35 / angles_trasm * 100
print(dThetaRis_trasm)

[0.00489317 0.00197468 0.00278447 0.00119245 0.00181777 0.00130629
 0.00193426 0.00145591]
[0.00611945 0.00475065 0.00326955]


## Statistical uncertainty

The error derived solely from the fit of the Compton peak can be taken in account with the formula:

$$
\delta_{fit} = \frac{\sigma_{fit}}{\sqrt{N}}
$$

Where N is the number of elements in my Compton peak, here as well one needs to express this error in terms of the scattering angle, the formula used is the same:

$$
\delta \theta_{fit} = \left| \frac{1}{sin^2 \theta \cdot (E')^2}\right| \cdot \delta_{fit}.
$$

In [58]:
dThetaFit = 1/(np.sin(angles)**2 * (energies**2)) * err_sigma / np.sqrt(count)
print(dThetaFit)

dThetaFit_trasm = 1/(np.sin(angles_trasm)**2 * (energies_trasm**2)) * err_sigma_trasm / np.sqrt(count_trasm)
print(dThetaFit_trasm)

[1.57231628e-07 6.43978028e-08 1.19067243e-07 4.74724502e-08
 1.38304779e-07 6.96607856e-08 1.82814040e-07 1.23204113e-07]
[1.35513929e-07 1.60857662e-07 1.16761642e-07]


## Final results

One can express the error bars on the x-axis as a sum of statistical and systematical uncertainties following the expression:

$$
\delta \theta = \delta \theta_{stat} \pm \delta \theta_{syst} = (\delta \theta_{fit}) \pm \left(\delta \theta_{peak} \oplus \delta \theta_{scattering} \oplus \delta \theta_{resolution} \right)
$$

In [ ]:
dThetaTOT = dThetaFit + np.sqrt(dThetaPeak**2 + dThetaRis**2 + dThetaScattering**2)
print (dThetaTOT)

dThetaTOT_trasm = dThetaFit_trasm + np.sqrt(dThetaPeak_trasm**2 + dThetaRis_trasm**2 + dThetaScattering**2)
print(dThetaTOT_trasm)

[0.06126201 0.06186401 0.06606664 0.07317939 0.08345257 0.09726168
 0.11546302 0.1393776 ]
[0.06270013 0.0612508  0.06191892]


In [60]:
np.savez('errori_arrays_riflex.npz', y_err=err_tot, x_err=dThetaTOT)
np.savez('errori_arrays_trasm.npz', y_err=err_tot_trasm, x_err=dThetaTOT_trasm)

print("File 'errori_arrays.npz' salvato con successo.")

File 'errori_arrays.npz' salvato con successo.
